# Julia programming language

Julia is compiled "just in time". This means that when you first run a piece of code it complies first and then runs it. This means that the first time your run a piece of code it will take longer than the next time you run it. Before we begin we will install all the packages we need to solve the ODE model, perform the paraemter esimtation, and visualise the results. Start this code at the start of the lecture as it may take some minutes to run. 

In [ ]:
# This code will activate the package environment by reading the Project.toml file in this directory
# and then instantiate the environment by installing all the packages in the environment.
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
# We will now load the packages that we will use in this notebook

# For solving differential equations
using OrdinaryDiffEq

# CairoMakie is a powerful plotting library that is built on top of Makie.jl
using CairoMakie

# Random is used for generating random numbers
using Random

# QuasiMonteCarlo is used for generating initial parameters with Latin Hypercube Sampling
using QuasiMonteCarlo

# Optimization is used for optimization
using Optimization

# For Profile Likelihood
using LikelihoodProfiler

# We also use some utility functions that will help us
include("utils/SelectionMethods.jl")


*Note:* If you get the following error message:
```julia
ERROR: LoadError: ArgumentError: Package XXX not found in current path
```

Please make sure that you have indeed installed the packages by running the cell above. If you have done so, please check in the bottom left corner of the Jupyter Notebook in
Visual Studio Code that you have selected the correct Julia environment. You can do this by clicking on the `Julia env` button and selecting the `VITAL_Trainschool_Prep` environment. Then, restart the Notebook by clicking on the `Restart` button on top of the Notebook.


# The Eindhoven Diabetes Education Simulator

The Eindhoven Diabetes Education Simulator (EDES) is physiology-based mathematical model describing the systemic interplay between glucose and insulin. The EDES model describes the rate of appeance of ingested glucose via the gut, the secretion of endogenous glucose from the liver, and both insulin dependent and independent uptake into the peripheral tissues. The EDES model also describes the secretion of insulin in response to changes in plasma glucose using a PD (proportional-derivative) controller. The EDES model consits of four ordinary differential equations describing rate of change of glucose in the stomach (gut_glucose) and plasma (plasma_glucose) and insulin in the plasma (plasma_insulin) and interstitial compartment (interstitial_insulin) and contains 13 kinetic paraemters that have been estimated from literature to describe the average OGTT response. More detail on the mode equations and parameters can be found in the file "EDES_Explained.pdf". 

<img src="figures/EDES.png" width="400" height="400">

## Implementing the EDES model in Julia

First we will implement the ODE equations. Note the implementation of these equations looks very similar to a model in Matlab or Python. 

A slight difference you will notice is that we also have the derivative of the state variables as an input to the function. This is because for efficiency, we typically define the ODE function as a *mutating* function. This means that the function will modify the given array `du` in place, rather than returning a new array. This is done to avoid unnecessary memory allocations. The exact details of memory allocations are not important for most users, but for future reference, this is a common pattern seen in developer-level Julia code.

In [ ]:
"""
edesode!(du, u, p, t)
  This function defines the system of ordinary differential equations (ODEs) that describe the EDES model.
"""
function edesode!(du, u, p, t)
  #specify initial values
    Ggut, Gpl, Ipl, Irem = u
    #specify the parameters
    k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, tau_i, tau_d, beta, Gren, EGPb, Km, f, Vg, c1, sigma, Dmeal, bw, Gb, Ib = p
  
    # Equation defining rate of decay of ingested glucose in stomach/gut
    du[1] = sigma * k1^sigma * t^(sigma-1) * exp(-(k1*t)^sigma) * Dmeal - k2 * Ggut
  
    # Equation describing the rate of change of plasma glucose
    gliv = EGPb - k3 * (Gpl - Gb) - k4 * beta * Irem #Endogenous glucose production in liver
    ggut = k2 * (f / (Vg * bw)) * Ggut #glucose appearing from meal
    u_ii =  EGPb * ((Km + Gb)/Gb) * (Gpl / (Km + Gpl)) # insulin independent uptake of glucose to peripheral tissues
    u_id = k5 * beta * Irem * (Gpl / (Km + Gpl)) # insulin dependent uptake of glucose to peripheral tissues
    u_ren = c1 / (Vg * bw) * (Gpl - Gren) * (Gpl > Gren) # renal excretion of excess glucose (>Gren threshold)
  
    du[2]  = gliv + ggut - u_ii - u_id - u_ren #full glucose in plasma equation
  
    #  Equation describing the rate of change of plasma insulin
    i_pnc = beta^(-1) * (k6 * (Gpl - Gb) + (k7 / tau_i) * Gb + k8 * tau_d * du[2]) #rate of insulin production in pancreas (funtion of glucose)
    i_liv = k7 * Gb * Ipl / (beta * tau_i * Ib) #rate of insulin degredation by liver
    i_int = k9 * (Ipl - Ib) #rate of insulin transport to interstitial compartment
    du[3] = i_pnc - i_liv - i_int # full insulin in plasma equation
    du[4] = i_int - k10 * Irem #full insulin in instertital compartment equation
end

The we will specify model inputs (body weight, fasting glucose and insulin values, and the amount of glucose in the meal), model parameter values (all kinetic parameters and model constants), and initial values (initial fasting values of each state variable) and build the ODE problem

In [ ]:
# specify physiology parameters
bw = 70.0 #body weight (kg)
Gb = 5.0 #fasting glucose (mmol/l)
Ib = 10.0 #fasting insulin (uIU/ml)

# meal parameters
Dmeal = 75.0e3 #amount of glucose in meal (mg)

# specify the EDES model parameters
k1 = 0.0105   # rate of glucose decay in stomach
k2 = 0.28     # rate of glucose transport from stomach to plasma
k3 = 6.07e-3  # rate of glucose effect on endogenous glucose production
k4 = 2.35e-4  # rate of insulin effect on endogenous glucose production
k5 = 0.0424   # rate of insulin dependent glucose uptake into peripheral tissues
k6 = 2.2975   # rate of insulin secretion (proportional to Gb)
k7 = 1.15     # rate of insulin secretion (integral of Gpl)
k8 = 7.27     # rate of insulin secretion (derivative of Gpl)
k9 = 3.83e-2  # delay parameter plasma to insterstitial insulin
k10 = 2.84e-1 # rate constant for degredation of insulin in remote compartment
tau_i = 31.0  # time delay integrator function
tau_d = 3.0   # time delay 
beta = 1.0    # conversion factor insulin
Gren = 9.0    # threshold for renal excretion of glucose
EGPb = 0.043  # basal rate of endogenous glucose production
Km = 13.2     # Michealis Menten coeficient for glucose uptake into periphery
f = 0.005551  #
Vg = 17.0 / bw # volume of distribution for glucose (function of body weight)
c1 = 0.1      # model constant
sigma = 1.4   # shape factor (appearance of meal)

# collection of parameters
p = [k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, tau_i, tau_d, beta, Gren, EGPb, Km, f, Vg, c1, sigma, Dmeal, bw, Gb, Ib]

# set initial conditions
u0 = [0.0, Gb, Ib, 0.0]

# set time span
tspan = (0.0, 240.0)

# define the ODE problem
prob = ODEProblem(edesode!, u0, tspan, p)

Finally, once we have defined the ODE problem and specified the model constants and parameters, and the intial values of each state variable we will solve the ODE problem and visualise the results. 

In [ ]:
#solve the ODE problem using the solve function
solution = solve(prob)

# visualize the solution
solution_figure = let f = Figure(size=(500,500))

    ax_g_gut = Axis(f[1,1], xlabel="Time [min]", ylabel="Glucose Mass [mg]", title="Gut Glucose")
    ax_g_plasma = Axis(f[1,2], xlabel="Time [min]", ylabel="Glucose Concentration [mM]", title="Plasma Glucose")
    ax_i_plasma = Axis(f[2,1], xlabel="Time [min]", ylabel="Insulin Concentration [mU/L]", title="Plasma Insulin")
    ax_i_int = Axis(f[2,2], xlabel="Time [min]", ylabel="Insulin Concentration [mU/L]", title="Interstitium Insulin")
  
    lines!(ax_g_gut, solution.t, solution[1,:], color=Makie.wong_colors()[1])
    lines!(ax_g_plasma, solution.t, solution[2,:], color=Makie.wong_colors()[1])
    lines!(ax_i_plasma, solution.t, solution[3,:], color=Makie.wong_colors()[1])
    lines!(ax_i_int, solution.t, solution[4,:], color=Makie.wong_colors()[1])
  
  
    f
  end

## Parameter Estimation for EDES

Previous work by Erdos et al. has identified a four parameter EDES model, estimaing k1, k5, k6, and k8 from plasma trajectories of glucose and insulin following a meal as being the most parsimonious for generating personlised EDES models from experimental data. 
|parameter|function|
|---------|-------------------------|
| k1 | rate of glucose appearence from stomach|
| k5 | rate of insulin dependent glucose uptake into tissues|
| k6 | rate of insulin secretion in response to plasma glucose (proportional)|
| k8 | rate of insulin secretion in response to plasma glucose (derivative) |

The remaining parameters are fixed to the population average estimated in the origianl study. Below we will set up the EDES model for parameter estimation by first defining a function that specifies model parameters. 

In [ ]:
#define a function that will specify parameters to be estimated and take in values for fixed parameters 
function construct_parameters(θ, c)

    # Estimated parameters
    k1 = θ[1]     # rate og glucose decay in stomach
    k5 = θ[2]     # rate of insulin dependent glucose uptake into peripheral tissues
    k6 = θ[3]     # rate of insulin secretion (proportional to Gb)
    k8 = θ[4]     # rate of insulin secretion (derivative of Gpl)
    
    # Fixed parameters
    k2 = c[1]     # rate of glcuose transport from stomach to plasma
    k3 = c[2]     # rate of glucose effect on endogenous glucose production
    k4 = c[3]     # rate of insulin effect on endogenous glucose production
    k7 = c[4]     # rate of insulin secretion (integral of Gpl)
    k9 = c[5]     # delay parameter plasma to insterstitial insulin
    k10 = c[6]    # rate constant for degredation of insulin in remote compartment
    tau_i = c[7]  # time delay integrator function
    tau_d = c[8]  # time delay 
    beta = c[9]   # conversion factor insulin
    Gren = c[10]  # threshold for renal excretion of glucose
    EGPb = c[11]  # basal rate of endogenous glucose production
    Km = c[12]    # Michealis Menten coeficient for glucose uptake into periphery
    f = c[13]     #
    Vg = c[14]    # volume of distribution for glucose (function of body weight)
    c1 = c[15]    # model constant
    sigma = c[16] # shape factor (appearance of meal)
    Dmeal = c[17] #amount of glucose in meal (mg)
    bw = c[18]    #body weight (kg)
    Gb = c[19]    #fasting glucose (mmol/l)
    Ib = c[20]    #fasting insulin (uIU/ml)
    return [k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, tau_i, tau_d, beta, Gren, EGPb, Km, f, Vg, c1, sigma, Dmeal, bw, Gb, Ib]
  end

We must also define a loss function. In this instance we define our loss function as the sum of square error between the measured plasma glucose and insulin concentrations and the model simulated glucose and insulin values for a given paramter set. To account for the difference in units, and as a results scale of values, between the glucose and insulin values we scale the error values by the maximum of their respective measurements. 

In [ ]:
#define our loss function
#
#loss function takes in values for the estimated parameter and a tuple containing the ODE problem name, values for the fixed parameters and model constants, and the measured experimetnal data. 
function loss(θ,(problem, constants ,data))
  #measured glucose and insulin data, including the sampling time points
    data_timepoints = data[1,:]
    glucose_data = data[2,:]
    insulin_data = data[3,:]
  

    #generate the full parameter vector by combining current values for the estimated parameters with the fixed values and constants.
    p = construct_parameters(θ, constants)

    # solve the ode problem for these parameter values
    pred = solve(problem, Tsit5(), p=p, saveat=data_timepoints, save_idxs=[2, 3], u0=[0.0, data[2,1], data[3,1], 0.0])
    sol = Array(pred)

    #calcualte the error between the measured glucose and insulin data and the EDES model simulation
    g_loss=(sol[1,:] - glucose_data)/maximum(glucose_data)
    i_loss=(sol[2,:] - insulin_data)/maximum(insulin_data)
    return sum(abs2, [g_loss; i_loss])
  end

Now that we have defined our loss function we can estimate values for k1, k5, k6, and k8 from plasma glucose and insulin values measured during an oral glucose tolerance test. This test involved the ingestion of solution containing 75g of glucose. Measured glucose and insulin values are specified below.  

In [ ]:
#import measured glcuose and insulin data
#meal plasma glucose and insulin values collected during an OGTT
data_glu = [5.4 7.1 8.6 8.9 8.5 7.6 6.6]
data_ins = [7.9 36.7 64.8 75.6 79.6 80.5 68.7]
data_time = [0 15 30 45 60 90 120]
data = [data_time;data_glu;data_ins]
#plasma glucose and insulin values for individual A
#data_glu = [4.42 5.1 6.2 6.1 5.1 4.7 4.5]
#data_ins = [4.4 27.3 59.8 78.1 61.0 46.6 40.6]
#data_time = [0 15 30 45 60 90 120]
#data = [data_time;data_glu;data_ins]

#sepecify the fixed parameter values
constants = [k2, k3, k4, k7, k9, k10, tau_i, tau_d, beta, Gren, EGPb, Km, f, Vg, c1, sigma, Dmeal, bw, data_glu[1], data_ins[1]];

# plot of the data
data_figure = let f = Figure(size=(400,200))

    ax_glu = Axis(f[1,1], xlabel="Time [min]", ylabel="Concentration [mM]", title="Glucose Data")
    ax_ins = Axis(f[1,2], xlabel="Time [min]", ylabel="Concentration [mU/L]", title="Insulin Data")
  
    scatter!(ax_glu, data[1,:], data[2,:], color=Makie.wong_colors()[1])
    scatter!(ax_ins, data[1,:], data[3,:], color=Makie.wong_colors()[1])
  
    f
  end

To estimate model parameters we are using the limited-memory BFGS alogorithm (LBFGS), which is an optimisation algorithm in the family of quasi-Newton methods that approximates the Broyden-Fletcher-GoldFarb-Shanno algorithm . The LBFGS is a local optimiser. Consequently we run this algorithm several times using different parameter initialisations. To ensure we have a good coverage of the parameter space we use a latin-hypercube design. 

We first start with defining a function to generate some initial parameter guesses. We first generate samples using LatinHyperCube Sampling and then we make use of the `SelectionMethods.jl` utility file for some flexibility if we want to perform some filtering on the initial guesses. For now, we leave it to the default, which is to return all the samples.

In [ ]:
function generate_initial_guesses(n_guesses, lower_bounds, upper_bounds; selection_method::SelectionMethod = SelectAll())

  initial_guesses = QuasiMonteCarlo.sample(n_guesses, lower_bounds, upper_bounds, LatinHypercubeSample())

  return evaluate_and_select(initial_guesses, selection_method)

end

Now we will perform the parameter generation and subsequent parameter estimation. The advantage of the speed of solvers in Julia means we can start with many parameter initialisations. Note how long it takes to run the parameter estimation proceedure for 1000 initial parameter sets

In [ ]:
# calcuate 1000 initial guesses for parameters being estimated using latin-hypercube design
initial_guess = generate_initial_guesses(1000, [0., 0., 0., 0.], [1., 0.5, 5., 20.])


results = []

optf = OptimizationFunction(loss, AutoForwardDiff())

# now run the local optimisation for each initial guess
for guess in eachcol(initial_guess)
    try
      res = solve(OptimizationProblem(optf, vec(guess), (prob, constants, data), lb = [0,0,0,0], ub=[1,0.5,10,25]), Optimization.LBFGS())
      push!(results, res)
    catch
      continue
    end
end;
# find the parameterisation with the lowest cost/objective funtion value
best_index = argmin([r.objective for r in results])
final = results[best_index].u

#solve the ODE problem using the solve function - to get the model simulation for this parameter set
solution = solve(prob, p =construct_parameters(final, constants), u0=[0.0, data[1,1], data[2,1], 0.0])

# visualize the solution against the data
solution_figure = let f = Figure(size=(500,500))

    ax_g_gut = Axis(f[1,1], xlabel="Time [min]", ylabel="Glucose Mass [mg/dL]", title="Gut Glucose")
    ax_g_plasma = Axis(f[1,2], xlabel="Time [min]", ylabel="Glucose Concentration [mM]", title="Plasma Glucose")
    ax_i_plasma = Axis(f[2,1], xlabel="Time [min]", ylabel="Insulin Concentration [mU/L]", title="Plasma Insulin")
    ax_i_int = Axis(f[2,2], xlabel="Time [min]", ylabel="Insulin Concentration [mU/L]", title="Interstitium Insulin")
  
    lines!(ax_g_gut, solution.t, solution[1,:], color=Makie.wong_colors()[1])
    lines!(ax_g_plasma, solution.t, solution[2,:], color=Makie.wong_colors()[1])
    lines!(ax_i_plasma, solution.t, solution[3,:], color=Makie.wong_colors()[1])
    lines!(ax_i_int, solution.t, solution[4,:], color=Makie.wong_colors()[1])

    scatter!(ax_i_plasma, data[1,:], data[3,:], color=Makie.wong_colors()[2])
    scatter!(ax_g_plasma, data[1,:], data[2,:], color=Makie.wong_colors()[2])
  
    f
end

As we estimated parameters using a high number of initial values we can also visualise the distrbution of the parameter estimates using a histogram. This can provide insights into our parameter space, indicating if all solutions converge to a single gloabl minimum or if multiple local minima exist for some parameters. There may also be a uniform distribution of solutions for some parameters suggesting we may not be able to idenitfy this parameter from the data. 

In [ ]:
# visualize the histogram of parameter estiamtes for all intiialiaslisations
k1=[]
k5=[]
k6=[]
k8=[]
for r =1:100
  push!(k1,results[r].u[1])
  push!(k5,results[r].u[2])
  push!(k6,results[r].u[3])
  push!(k8,results[r].u[4])
end

solution_figure = let f = Figure(size=(500,500))

    ax_k1 = Axis(f[1,1], xlabel="k1 value", title="k1 distribution")
    ax_k5 = Axis(f[1,2], xlabel="k5 value", title="k5 distribution")
    ax_k6 = Axis(f[2,1], xlabel="k6 value", title="k6 distribution")
    ax_k8 = Axis(f[2,2], xlabel="k8 value", title="k8 distribution")

    hist!(ax_k1, k1, bins=10,color=Makie.wong_colors()[1])
    hist!(ax_k5, k5, bins=10,color=Makie.wong_colors()[1])
    hist!(ax_k6, k6, bins=10,color=Makie.wong_colors()[1])
    hist!(ax_k8, k8, bins=10,color=Makie.wong_colors()[1])
  
    f
  end

What does the distribution of the parameter estimates across the 1000 initital guesses tell us? is there a parameter where all estimates converge to the same local minima or are there multiple local minima? Is there a parameter with a uniform distribution? What would this mean for parameter identifiability? 

## Implementation of Profile Likelihood Analysis

The manual implementation of Profile Likelihood Analysis requires some tricky functions, but it is not too difficult to implement. We will take you through the steps of implementing the Profile Likelihood Analysis for the EDES model.

First define a nice struct that contains the output of a PLA. We will fill this struct later.

In [ ]:
struct PLAResult
    likelihood_values
    parameter_values
    optim_index::Int
    other_parameter_values
end

The loss function also needs some modification. We define a constructor for modifying the loss function which returns a new loss function. This new loss function will compute the loss when fixing one of the parameters to a certain value. This is the essence of the Profile Likelihood Analysis.

In [ ]:
function loss_pla(loss, pla_param, fixed_parameter_value, npar)

    # compute the order of the parameters
    parameter_order = zeros(Int64,npar)
    parameter_order[[1:pla_param-1; (pla_param+1):npar]] .= 1:npar-1
    parameter_order[pla_param] = npar
    
    function _loss_pla(θ, p)
      # construct the full parameter vector
      θ_full = [θ; fixed_parameter_value][parameter_order]
      loss(θ_full, p)
    end
end

Each PLA step then involves fixing the parameter value to a certain value, defining the loss function to use that fixed value, and then running the optimisation algorithm to find the best parameters for the other parameters. We also define a constructor for doing this, which returns a new function that will run a PLA step.

In [ ]:
function pla_step(pla_param_index, loss, θ_init, lb, ub, p)
    npar = length(θ_init)
    lb_pla = [lb[1:pla_param_index-1]; lb[(pla_param_index+1):npar]]
    ub_pla = [ub[1:pla_param_index-1]; ub[(pla_param_index+1):npar]]
    initial_guess = [θ_init[1:pla_param_index-1]; θ_init[pla_param_index+1:end]]

    function _likelihood(pla_param)
        loss_function = loss_pla(loss, pla_param_index, pla_param, length(initial_guess)+1)
        optfunc = OptimizationFunction(loss_function, Optimization.AutoForwardDiff())
        optprob = OptimizationProblem(optfunc, initial_guess, p, lb = lb_pla, ub = ub_pla)

        optsol = solve(optprob, Optimization.LBFGS())
        optsol.objective, optsol.u
    end
end

Finally, we combine this into a function that will run such a PLA step for a parameter iteratively over a range of values and stores the results in the struct we have defined earlier.

In [ ]:
function run_pla(pla_param, param_range, param_optim, loss, initial_guess, lb_pla, ub_pla, p)

    # setup PLA likelihood
    step = pla_step(
        pla_param, 
        loss, 
        initial_guess, 
        lb_pla, ub_pla, p
    )

    param_range = sort(unique([param_range; param_optim]))
    optim_index = findfirst(x -> x == param_optim, param_range)
    pla_likelihood_values = Float64[]
    other_parameter_values = typeof(initial_guess)[]
    parameter_values = typeof(param_optim)[]
    for px in param_range
        try
            l, _p = step(px)
            push!(pla_likelihood_values, l)
            push!(other_parameter_values, _p)
            push!(parameter_values, px)
        catch e
            throw(e)
            continue
        end
    end
    
    return PLAResult(pla_likelihood_values, parameter_values, optim_index, other_parameter_values)
end

If we have a nice struct, we can define a function to plot the results of a PLA run for one parameter.

In [ ]:
function likelihood_profile!(ax, pla_result::PLAResult, param_index, param_name)
    parameter_values =  pla_result.parameter_values
    likelihood_values = pla_result.likelihood_values
    lines!(ax, parameter_values, likelihood_values, color=Makie.wong_colors()[1])
    #scatter!(ax, parameter_values, likelihood_values, color=Makie.wong_colors()[2])    
end

## Running the PLA for the EDES model

Now we are ready to run PLA for each parameter. We have configured suitable ranges for each parameter.

In [ ]:
# configure pla runs for each parameter
ranges = [
    range(1e-5, 0.05, length=800),
    range(0.001, 0.5, length=800),
    range(1.0, 12.0, length=400),
    range(1e-8, 5.0, length=400)
]

pla_results = [
    run_pla(i, ranges[i], final[i], loss, final, [0.0, 0.0, 0.0, 0.0], [1.0, 0.5, 10.0, 25.0], (prob, constants, data)) for i in eachindex(final)
];

We can now visualize the results of the PLA for each parameter.

In [ ]:
figure_likelihood = let f = Figure()
    positions = [(1,1), (1,2), (2,1), (2,2)]
    for (i, (pos, param_name)) in enumerate(zip(positions, ["k1", "k5", "k6", "k8"]))
        ax = Axis(getindex(f, pos...), xlabel=param_name, ylabel="Likelihood", title="Profile Likelihood for $param_name")
        likelihood_profile!(ax, pla_results[i], i, param_name)
        scatter!(ax, [final[i]], [results[best_index].objective], color=Makie.wong_colors()[2])
    end

    f
end

Are the parameters in the four-parameter EDES model identifiable? Which parameters are non-identifiable? Are they structurally or practically non-identifiable? Is this the same for the average case and the two individuals? What could be done to resolve this non-identifiability?

### Exercise
Fix k8 = 7.25. What is the effect on the parameter estimation? What is the effect on the PLA? Are the paramters now identifiable? 

# Extending EDES with data from wearable sensors

For most individual the three parameter EDES modlel was identifiable given seven venous measurments of glucose and insulin. However, for some individuals, particulary those with faster glucose responses the discrete venous sampling in the standard OGTT often missed essential dynamics and not all parameters were practically non-identifiable. In recent years there has been considerable advancements in wearable flash glucose or continous glucose monitors, which quantify interstitial glucose levels every five mintes. This more freqently sampling has the potential to resolve the issues we have with practical non-identifiabiliy. 

### Exercise
Modify the parameter estimation procedure for the EDES model to work with the CGM glucose data provided below. (Hint: adjust the error function to be dependant only on CGM glucose values - no insulin). Evalute is the EDES model parameters are identifiable using only the CGM data. 

In [ ]:
# mean venous glucose and invsulin values measured during an OGTT
venous_glu = [5.4 7.1 8.6 8.9 8.5 7.6 6.6]
venous_ins = [7.9 36.7 64.8 75.6 79.6 80.5 68.7]
venous_time = [0 15 30 45 60 90 120]
venous_data = [venous_time;venous_glu;venous_ins]
#mean intersitial glucose measured using a CGM device during an OGTT
CGM_glu = [6.1 6.2 6.4 6.7 7.1 7.6 8.1 8.5 8.9 9.2 9.4 9.5 9.6 9.6 9.5 9.4 9.3 9.2 9.0 8.9 8.8 8.6 8.5 8.3 8.2 7.9]
CGM_time = [0	5	10	15	20	25	30	35	40	45	50	55	60	65	70	75	80	85	90	95	100	105	110	115	120	125]
CGM_data = [CGM_data;CGM_glucose]